In [1]:
import sys, time, re
import os
import pandas as pd
import pylab as plt
from importlib import reload
%matplotlib inline
import itertools as it
import numpy as np
from multiprocessing import Pool as ThreadPool
import  Levenshtein
from tqdm import tqdm_notebook as tqdm
import pickle
import gc
import sys
sys.path.append('../scripts')

import ml_utils as utils
reload(utils)

<module 'ml_utils' from '../scripts/ml_utils.py'>

Generator name simila

### Generate true mapping

In [ ]:
utils.generate_true_mapping(from_raw=False)

### Read graphs

In [2]:
%%time
g1_fname = 'vk_lid_rid.csv'
g2_fname = 'inst_lid_rid.csv'
G1 = utils.read_g(g1_fname)
G2 = utils.read_g(g2_fname)

CPU times: user 2.52 s, sys: 76 ms, total: 2.59 s
Wall time: 2.56 s


### Read matches and start ML step

In [3]:
threshold=91
is_repeat = True
matches_file_name = 'matches_s_01_th_091_t_10-12_13:19.pickle'

matches = utils.read_matches(matches_file_name, threshold, is_repeat)
utils.precision_recall(matches)

../matches/repeat/091/matches_s_01_th_091_t_10-12_13:19.pickle
matches len 4765
(78897392, 608311198)
(20158885, 482179932)
(12852529, 294223340)
(239645190, 53256746)
(134652490, 4145166609)


(0.9565582371458552, 0.18484123443773065)

### 1 step

In [ ]:
reload(utils)
utils.gen_train_data(matches, G1, G2, save_to = 'train_data_matches', threads = 1)

In [4]:
features, labels = utils.load_train_data('train_data_matches')
len(features), len(labels)

(11503, 11503)

### Generate 2 step test data (data/generate/test2_predicted.pickle)

In [19]:
df_l_others = utils.read_clean_csv(fname='vk_personal2.csv', from_raw = False)
df_l_others.rename(columns={'uid_vk': 'uid'}, inplace=True)
df_r_others = utils.read_clean_csv(fname='inst_personal.csv', from_raw = False)
df_r_others.rename(columns={'uid_inst': 'uid'}, inplace=True)

In [37]:
df_l_others_90, df_r_others_90 = utils.filter_others_from_predicted(df_l_others, df_r_others, matches)

26460 21695 4765 26460


In [40]:
reload(utils)
name_sim_threshold = 90
results_90, lid_rid_90 = utils.ml_iteration(df_l_others_90, df_r_others_90, forest, G1, G2, 
                    name_sim_threshold, threads = 6, save_to='lid_rid_90')

Cache loaded 24800 20794
Start 0


HBox(children=(IntProgress(value=0, max=3615), HTML(value='')))

Start 1


HBox(children=(IntProgress(value=0, max=3615), HTML(value='')))

Start 2


...

Start 4


HBox(children=(IntProgress(value=0, max=3615), HTML(value='')))

Start 5


HBox(children=(IntProgress(value=0, max=3620), HTML(value='')))


Thread 3 True = 4538 False = 19300 Total = 23839 True/Total = 0.1903603339066236

Thread 5 True = 4252 False = 19875 Total = 24128 True/Total = 0.17622679045092837

Thread 4 True = 4527 False = 20514 Total = 25042 True/Total = 0.18077629582301732

Thread 2 True = 6291 False = 18912 Total = 25204 True/Total = 0.2496032375813363

Thread 0 True = 5994 False = 21103 Total = 27098 True/Total = 0.2211971363200236

Thread 1 True = 6938 False = 21336 Total = 28275 True/Total = 0.2453757736516357
0 true count 5994
1 true count 6938
2 true count 6291
3 true count 4538
4 true count 4527
5 true count 4252
Clear cache 0 0
resulting 2 step 1700 from 32540  |  0.05224339274738783
lid_rid 1700 1700 1700


In [41]:
utils.precision_recall(lid_rid_90)

(0.39588235294117646, 0.027292266515268258)

### Generate 3 step test data (data/generate/test3_data.pickle)

In [ ]:
def prepare_data_for_threads(df_l, df_r, model, G1, G2, threads, name_sim_threshold = None):
    data = []

    thr_size = len(df_l) // threads
    for i in range(threads):
        s = i * thr_size
        e = i * thr_size + thr_size
        d = {
            'thread_num': i,
            'vals_l': df_l[s:e] if i + 1 < threads else df_l[s:],
            'vals_r': df_r,
            'model': model,
            'G1': G1,
            'G2': G2,
        }
        if name_sim_threshold:
            d['name_sim_threshold'] =  name_sim_threshold
        data.append(d)
    return datareshape((1, -1)

In [ ]:
reload(utils)

def collect_true(data):
    df_vals_l = data['vals_l']
    df_vals_r = data['vals_r']
    model = data['model']
    G1 = data['G1']
    G2 = data['G2']
    thread_num = data['thread_num']

    res_list = []
    lev = Levenshtein
    print('Start', thread_num)
    true_count = 0
    for row_l in tqdm(df_vals_l):
        for row_r in tqdm(df_vals_r):
#             ratio = lev.ratio(row_l[1], row_r[1])
            x = np.array(utils.feature(G1, row_l, G2, row_r)).reshape((1, -1))
            if model.predict(x) == 1:
                true_count += 1
                res_list.append((row_l, row_r)) # ratio
    total_count = len(df_vals_l) * len(df_vals_r)
    print('Thread', thread_num, 'True =', true_count, 'False =', total_count - true_count,
          'Total =', total_count, 'True/Total =', true_count / total_count)
    return res_list

def get_test_3_step_palalel(true_1step, true_2step, model, G1, G2, threads=4):
    
    
    filter_g = lambda G, col_num: list(set(G.nodes()) - (set((x[col_num] for x in true_1step)) | 
                                                      set((x[col_num] for x in true_2step))))
    df_l = filter_g(G1, 0)
    df_r = filter_g(G2, 1)
    data = utils.prepare_data_for_threads(df_l, df_r, forest, G1, G2, threads=threads)
    with ThreadPool(threads) as pool:
        results = pool.map(collect_true, data)
    439850
    for i, res in enumerate(results):
        print(i, 'true count', len(res))
    pickle.dump(results, open(os.path.join(utils.folder_gen, 'test3_predicted.pickle'), "wb"))
    utils.f_set1s = dict()
    utils.f_set2s = dict()
    return results

get_test_3_step_palalel(matched_1step, results2, forest, G1, G2, threads=5)
gc.collect()

### Name threshold 82

In [ ]:
lid_rid_90 = pickle.load(open(os.path.join(utils.folder_gen, 'lid_rid_90.pickle'), "rb"))
# ToDo retrain model there
df_l_others_80, df_r_others_80 = filter_others_from_predicted(vk_others, inst_others, lid_rid_90)

lid_rid_80 = ml_iteration(df_l_others, df_r_others, forest, G1, G2, 
                    name_sim_threshold = 0.8, threads = 6, save_to = 'lid_rid_80')

gc.collect()

In [ ]:
utils.gen_train_data(lid_rid_80, G1, G2, save_to = 'train_data_80')

In [ ]:
utils.precision(lid_rid_80)

In [ ]:
from scipy.stats import ks_2samp

def dist_sim(G1, n, G2, m):
    sl = [v.degree() for v in vl.neighbors()]
    sr = [v.degree() for v in vr.neighbors()]
    return ks_2samp(sl, sr)

### Name threashold 70

In [ ]:
lid_rid_80 = pickle.load(open(os.path.join(utils.folder_gen, 'lid_rid_80.pickle'), "rb"))

model_70 = utils.retrain_model(model_80, ['train_data_matched', 'train_data_90', 'train_data_80', ])

df_l_others_70, df_r_others_70 = utils.filter_others_from_predicted(df_l_others_80, df_r_others_80, lid_rid_80)

lid_rid_70 = utils.ml_iteration(df_l_others_70, df_r_others_70, model_70, G1, G2, 
                    name_sim_threshold = 0.7, threads = 6, save_to = 'lid_rid_70')

gc.collect()

In [ ]:
utils.gen_train_data(lid_rid_70, G1, G2, save_to = 'train_data_70')

In [ ]:
utils.precision(lid_rid_70)

In [ ]:
def overall_precision():
    ls = []
    for t in (6,7,8,9):
        lid_rid = pickle.load(open(os.path.join(utils.folder_gen, 'lid_rid_%d0.pickle' % t), "rb"))
        ls += lid_rid
        
    ls += [(x[0], x[1]) for x in pd.read_csv(os.path.join(utils.folder_gen, 'true_mapping.csv'))[:2500].values.tolist()]
    utils.precision(ls)
    
overall_precision()

#### Features cache

In [ ]:
utils.f_set1s = dict(pickle.load(open(os.path.join(utils.folder_gen, 'features_G1.pickle'), "rb")))
utils.f_set2s = dict(pickle.load(open(os.path.join(utils.folder_gen, 'features_G2.pickle'), "rb")))

### Name threashold 60

In [ ]:
lid_rid_70 = pickle.load(open(os.path.join(utils.folder_gen, 'lid_rid_70.pickle'), "rb"))

model_60 = utils.retrain_model(model_70, ['train_data_matched', 'train_data_90', 'train_data_80', 'train_data_70'])

df_l_others_60, df_r_others_60 = utils.filter_others_from_predicted(df_l_others_70, df_r_others_70, lid_rid_70)

lid_rid_60 = utils.ml_iteration(df_l_others_60, df_r_others_60, model_60, G1, G2, 
                    name_sim_threshold = 0.6, threads = 6, save_to = 'lid_rid_60')

gc.collect()

In [ ]:
utils.precision(lid_rid_60)

### Feature generation for ML on 1 step

In [ ]:
%%time

features, labels = utils.gen_train_data_1step('step1_data.pickle', from_raw=False)

In [ ]:
for a,b in it.islice(zip(features, labels), 2):
    print(a, b)

### ML models

In [26]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc

In [27]:
x_train, x_test, y_train, y_test = train_test_split(
     features, labels, test_size=0.3, random_state=42)
# X_train, y_train = features, labels

In [30]:
forest = RandomForestClassifier(n_estimators = 400, criterion ='gini',
                        bootstrap = True, max_features = 'auto' ,n_jobs=4)
forest.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=4,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [31]:
forest.score(x_test, y_test)

0.74007534048101997

In [32]:
gbc = GradientBoostingClassifier(n_estimators=100)
gbc.fit(x_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [33]:
gbc.score(x_test, y_test)

0.7258765575195596

In [ ]:
y_score = forest.predict(X_test)
y_true = y_test

fpr, tpr, thresholds = roc_curve(y_true, y_score)
roc_auc = auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b',
label='GS, AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

### Try keras ML

In [24]:
from keras.losses import mean_squared_error
from keras.models import Sequential, Model
from keras.layers import LSTM,Activation,Dense, Lambda, Input, Concatenate, Add, Reshape, Dropout
from keras.optimizers import Adam
from keras.metrics import mean_squared_error
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras.engine.topology import Layer
from keras.initializers import RandomNormal
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import multiply
from keras import backend as K
from keras.constraints import non_neg
import tensorflow as tf
import keras

In [6]:
#Формирование модели
model_number = 0
best_weights_filepath = './best_weights.hdf5'
log_path = './logs_nn/'

In [13]:
def get_model(x_train,
              y_train,
              model_number=0,
              best_weights_filepath=best_weights_filepath,
              log_path='./logs_nn/'):
    '''hyperparams'''
#     learning_rate = 1e-3
#     reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=50, min_lr=0.0001)
#     optimiser = Adam(lr=learning_rate)
    dense_activation = 'relu'
    dense_1layer_nn_count_coef = 1
    dense_2layer_nn_count_coef = 0.5
    
    out_len = 1
    feature_amount = len(x_train[0])
    in_shape = (feature_amount, )
    
    ''' 1. Вход '''
    inputs = Input(shape=in_shape, name="in_data")

    inputs_transformed = BatchNormalization()(inputs)
    print('inputs_transformed', inputs_transformed.shape)
    ''' 6. Несколько полносвязных слоев '''
    dense_nn_count = int(dense_1layer_nn_count_coef * feature_amount)
    dense_model = Dense(units = dense_nn_count, activation = dense_activation)(inputs_transformed)
    print(dense_model.shape)
    dropout = Dropout(0.2)(dense_model)
    dense_nn_count = int(dense_2layer_nn_count_coef * feature_amount)
    dense_model = Dense(units = dense_nn_count, activation = dense_activation)(dense_model)
    print(dense_model.shape)
    output = Dense(units = out_len, activation = 'sigmoid', name='out_states')(dense_model)   
    print(dense_model.shape)
    
    model = Model(inputs, output)
    
    tb_callback = TensorBoard(log_dir=log_path + '/', histogram_freq=0, write_graph=True,
                              write_images=True)
    bm_callback = ModelCheckpoint(best_weights_filepath, monitor='val_loss', verbose=1, save_best_only=True,
                                  mode='auto')
    
    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy', 'binary_crossentropy'])
    return [model, [bm_callback, tb_callback]]

In [14]:
!rm -rf ./logs_nn/

In [16]:
[model, callbacks] = get_model(x_train, y_train, log_path = log_path+str(model_number))

inputs_transformed (?, 85)
(?, 85)
(?, 42)
(?, 42)


In [18]:
batch_size = 32
epochs = 1200

model.fit(x_train, y_train,
      batch_size=batch_size,
      epochs=epochs,
      validation_data=(x_test, y_test),
      initial_epoch = 0,
      callbacks = callbacks,
      verbose = 1
     )

Train on 8052 samples, validate on 3451 samples
Epoch 1/1200
8052/8052 [==============================] - 0s - loss: 0.5553 - acc: 0.7016 - binary_crossentropy: 0.5553 - val_loss: 0.5488 - val_acc: 0.7076 - val_binary_crossentropy: 0.5488
Epoch 2/1200
8052/8052 [==============================] - 0s - loss: 0.5237 - acc: 0.7291 - binary_crossentropy: 0.5237 - val_loss: 0.5425 - val_acc: 0.7195 - val_binary_crossentropy: 0.5425
Epoch 3/1200
8052/8052 [==============================] - 0s - loss: 0.5125 - acc: 0.7389 - binary_crossentropy: 0.5125 - val_loss: 0.5314 - val_acc: 0.7282 - val_binary_crossentropy: 0.5314
Epoch 4/1200
8052/8052 [==============================] - 0s - loss: 0.5008 - acc: 0.7460 - binary_crossentropy: 0.5008 - val_loss: 0.5250 - val_acc: 0.7230 - val_binary_crossentropy: 0.5250
Epoch 5/1200
8052/8052 [==============================] - 0s - loss: 0.4915 - acc: 0.7507 - binary_crossentropy: 0.4915 - val_loss: 0.5224 - val_acc: 0.7343 - val_binary_crossentropy: 0.52

8052/8052 [==============================] - 0s - loss: 0.4240 - acc: 0.7966 - binary_crossentropy: 0.4240 - val_loss: 0.5393 - val_acc: 0.7389 - val_binary_crossentropy: 0.5393
Epoch 24/1200
8052/8052 [==============================] - 0s - loss: 0.4170 - acc: 0.8008 - binary_crossentropy: 0.4170 - val_loss: 0.5440 - val_acc: 0.7337 - val_binary_crossentropy: 0.5440
Epoch 25/1200
8052/8052 [==============================] - 0s - loss: 0.4185 - acc: 0.8018 - binary_crossentropy: 0.4185 - val_loss: 0.5474 - val_acc: 0.7372 - val_binary_crossentropy: 0.5474
Epoch 26/1200
8052/8052 [==============================] - 0s - loss: 0.4116 - acc: 0.8046 - binary_crossentropy: 0.4116 - val_loss: 0.5588 - val_acc: 0.7270 - val_binary_crossentropy: 0.5588
Epoch 27/1200
8052/8052 [==============================] - 0s - loss: 0.4117 - acc: 0.8027 - binary_crossentropy: 0.4117 - val_loss: 0.5377 - val_acc: 0.7366 - val_binary_crossentropy: 0.5377
Epoch 28/1200
8052/8052 [=============================

8052/8052 [==============================] - 0s - loss: 0.3638 - acc: 0.8311 - binary_crossentropy: 0.3638 - val_loss: 0.5559 - val_acc: 0.7334 - val_binary_crossentropy: 0.5559
Epoch 48/1200
8052/8052 [==============================] - 0s - loss: 0.3584 - acc: 0.8332 - binary_crossentropy: 0.3584 - val_loss: 0.5778 - val_acc: 0.7201 - val_binary_crossentropy: 0.5778
Epoch 49/1200
8052/8052 [==============================] - 0s - loss: 0.3593 - acc: 0.8366 - binary_crossentropy: 0.3593 - val_loss: 0.5942 - val_acc: 0.7270 - val_binary_crossentropy: 0.5942
Epoch 50/1200
8052/8052 [==============================] - 0s - loss: 0.3584 - acc: 0.8377 - binary_crossentropy: 0.3584 - val_loss: 0.5968 - val_acc: 0.7320 - val_binary_crossentropy: 0.5968
Epoch 51/1200
8052/8052 [==============================] - 0s - loss: 0.3536 - acc: 0.8381 - binary_crossentropy: 0.3536 - val_loss: 0.6084 - val_acc: 0.7331 - val_binary_crossentropy: 0.6084
Epoch 52/1200
8052/8052 [=============================

8052/8052 [==============================] - 0s - loss: 0.3240 - acc: 0.8527 - binary_crossentropy: 0.3240 - val_loss: 0.6469 - val_acc: 0.7172 - val_binary_crossentropy: 0.6469
Epoch 72/1200
8052/8052 [==============================] - 0s - loss: 0.3244 - acc: 0.8587 - binary_crossentropy: 0.3244 - val_loss: 0.6427 - val_acc: 0.7279 - val_binary_crossentropy: 0.6427
Epoch 73/1200
8052/8052 [==============================] - 0s - loss: 0.3231 - acc: 0.8583 - binary_crossentropy: 0.3231 - val_loss: 0.6415 - val_acc: 0.7215 - val_binary_crossentropy: 0.6415
Epoch 74/1200
8052/8052 [==============================] - 0s - loss: 0.3189 - acc: 0.8585 - binary_crossentropy: 0.3189 - val_loss: 0.6472 - val_acc: 0.7215 - val_binary_crossentropy: 0.6472
Epoch 75/1200
8052/8052 [==============================] - 0s - loss: 0.3139 - acc: 0.8608 - binary_crossentropy: 0.3139 - val_loss: 0.6623 - val_acc: 0.7244 - val_binary_crossentropy: 0.6623
Epoch 76/1200
8052/8052 [=============================

8052/8052 [==============================] - 0s - loss: 0.2961 - acc: 0.8710 - binary_crossentropy: 0.2961 - val_loss: 0.7489 - val_acc: 0.7160 - val_binary_crossentropy: 0.7489
Epoch 96/1200
8052/8052 [==============================] - 0s - loss: 0.2919 - acc: 0.8730 - binary_crossentropy: 0.2919 - val_loss: 0.6952 - val_acc: 0.7210 - val_binary_crossentropy: 0.6952
Epoch 97/1200
8052/8052 [==============================] - 0s - loss: 0.2866 - acc: 0.8778 - binary_crossentropy: 0.2866 - val_loss: 0.7181 - val_acc: 0.7204 - val_binary_crossentropy: 0.7181
Epoch 98/1200
8052/8052 [==============================] - 0s - loss: 0.2843 - acc: 0.8710 - binary_crossentropy: 0.2843 - val_loss: 0.7185 - val_acc: 0.7224 - val_binary_crossentropy: 0.7185
Epoch 99/1200
8052/8052 [==============================] - 0s - loss: 0.2932 - acc: 0.8706 - binary_crossentropy: 0.2932 - val_loss: 0.7195 - val_acc: 0.7163 - val_binary_crossentropy: 0.7195
Epoch 100/1200
8052/8052 [============================

...

8052/8052 [==============================] - 0s - loss: 0.1760 - acc: 0.9334 - binary_crossentropy: 0.1760 - val_loss: 1.2232 - val_acc: 0.7007 - val_binary_crossentropy: 1.2232
Epoch 432/1200
8052/8052 [==============================] - 0s - loss: 0.1795 - acc: 0.9297 - binary_crossentropy: 0.1795 - val_loss: 1.2336 - val_acc: 0.6940 - val_binary_crossentropy: 1.2336
Epoch 433/1200
8052/8052 [==============================] - 0s - loss: 0.1744 - acc: 0.9322 - binary_crossentropy: 0.1744 - val_loss: 1.2034 - val_acc: 0.7053 - val_binary_crossentropy: 1.2034
Epoch 434/1200
8052/8052 [==============================] - 0s - loss: 0.1753 - acc: 0.9285 - binary_crossentropy: 0.1753 - val_loss: 1.2555 - val_acc: 0.7062 - val_binary_crossentropy: 1.2555
Epoch 435/1200
8052/8052 [==============================] - 0s - loss: 0.1834 - acc: 0.9293 - binary_crossentropy: 0.1834 - val_loss: 1.2291 - val_acc: 0.6937 - val_binary_crossentropy: 1.2291
Epoch 436/1200
8052/8052 [========================

8052/8052 [==============================] - 0s - loss: 0.1781 - acc: 0.9321 - binary_crossentropy: 0.1781 - val_loss: 1.2422 - val_acc: 0.6917 - val_binary_crossentropy: 1.2422
Epoch 456/1200
8052/8052 [==============================] - 0s - loss: 0.1820 - acc: 0.9317 - binary_crossentropy: 0.1820 - val_loss: 1.2477 - val_acc: 0.6949 - val_binary_crossentropy: 1.2477
Epoch 457/1200
8052/8052 [==============================] - 0s - loss: 0.1786 - acc: 0.9313 - binary_crossentropy: 0.1786 - val_loss: 1.2578 - val_acc: 0.6995 - val_binary_crossentropy: 1.2578
Epoch 458/1200
8052/8052 [==============================] - 0s - loss: 0.1743 - acc: 0.9328 - binary_crossentropy: 0.1743 - val_loss: 1.2597 - val_acc: 0.6978 - val_binary_crossentropy: 1.2597
Epoch 459/1200
8052/8052 [==============================] - 0s - loss: 0.1795 - acc: 0.9318 - binary_crossentropy: 0.1795 - val_loss: 1.2537 - val_acc: 0.7044 - val_binary_crossentropy: 1.2537
Epoch 460/1200
8052/8052 [========================

8052/8052 [==============================] - 0s - loss: 0.1714 - acc: 0.9303 - binary_crossentropy: 0.1714 - val_loss: 1.3016 - val_acc: 0.6847 - val_binary_crossentropy: 1.3016
Epoch 480/1200
8052/8052 [==============================] - 0s - loss: 0.1765 - acc: 0.9333 - binary_crossentropy: 0.1765 - val_loss: 1.2479 - val_acc: 0.6876 - val_binary_crossentropy: 1.2479
Epoch 481/1200
8052/8052 [==============================] - 0s - loss: 0.1777 - acc: 0.9295 - binary_crossentropy: 0.1777 - val_loss: 1.2416 - val_acc: 0.6986 - val_binary_crossentropy: 1.2416
Epoch 482/1200
8052/8052 [==============================] - 0s - loss: 0.1720 - acc: 0.9359 - binary_crossentropy: 0.1720 - val_loss: 1.2791 - val_acc: 0.6949 - val_binary_crossentropy: 1.2791
Epoch 483/1200
8052/8052 [==============================] - 0s - loss: 0.1684 - acc: 0.9354 - binary_crossentropy: 0.1684 - val_loss: 1.3178 - val_acc: 0.6914 - val_binary_crossentropy: 1.3178
Epoch 484/1200
8052/8052 [========================

8052/8052 [==============================] - 0s - loss: 0.1830 - acc: 0.9308 - binary_crossentropy: 0.1830 - val_loss: 1.2541 - val_acc: 0.6905 - val_binary_crossentropy: 1.2541
Epoch 504/1200
8052/8052 [==============================] - 0s - loss: 0.1782 - acc: 0.9311 - binary_crossentropy: 0.1782 - val_loss: 1.2892 - val_acc: 0.6879 - val_binary_crossentropy: 1.2892
Epoch 505/1200
8052/8052 [==============================] - 0s - loss: 0.1724 - acc: 0.9309 - binary_crossentropy: 0.1724 - val_loss: 1.2731 - val_acc: 0.6966 - val_binary_crossentropy: 1.2731
Epoch 506/1200
8052/8052 [==============================] - 0s - loss: 0.1664 - acc: 0.9331 - binary_crossentropy: 0.1664 - val_loss: 1.2615 - val_acc: 0.7010 - val_binary_crossentropy: 1.2615
Epoch 507/1200
8052/8052 [==============================] - 0s - loss: 0.1823 - acc: 0.9292 - binary_crossentropy: 0.1823 - val_loss: 1.2858 - val_acc: 0.7018 - val_binary_crossentropy: 1.2858
Epoch 508/1200
8052/8052 [========================

KeyboardInterrupt: 